In [1]:
import numpy as np
import copy
from sklearn.neural_network import MLPClassifier
from numpy.random import *
from operator import itemgetter

In [2]:
def inputFieldData(text_path):
    f = open(text_path)
    line = f.readline()
    line_count = 1
    field_size = int(len(line) / 2)
    field = np.empty((0,field_size),int)
    agents_pos_list = [[5,2],[1,1],[6,1]]
    goal_pos_list = [[1,6],[6,6],[3,3]]
    while line:
        last_idx = line.find("\n")
        field_row = np.array([])
        for i in range(0,last_idx,2):
            field_row = np.append(field_row,int(line[i]))
        field = np.append(field,np.array([field_row]),axis=0)
        line = f.readline()
        line_count += 1
    f.close()
    return field,agents_pos_list,goal_pos_list

In [3]:
field,agents_pos_list,goal_pos_list = inputFieldData("singleAgentField2.txt")
print(field)
print("agents_pos_list = {0}".format(agents_pos_list))

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
agents_pos_list = [[5, 2], [1, 1], [6, 1]]


In [4]:
field_row = len(field[0])
field_col = len(field)

In [5]:
alpha = 0.1
q_gamma = 0.9
now_pos_ground = 0

## aboutAction
action=0 : ↑
action=1 : →
action=2 : ↓
action=3 : ←
action=4 : .

In [235]:
def jumpToNextPos(pos,action):
    if action == 0:
        next_pos = [pos[0]-1,pos[1]]
    elif action == 1:
        next_pos = [pos[0],pos[1]+1]
    elif action == 2:
        next_pos = [pos[0]+1,pos[1]]
    elif action == 3:
        next_pos = [pos[0],pos[1]-1]
    else:
        next_pos = [pos[0],pos[1]]
    return next_pos

In [236]:
def checkPosition(pos,action,field):
    r = action
    next_pos = jumpToNextPos(pos,action)
    while field[next_pos[0],next_pos[1]] == 1:
        r = int(rand()*5)
        while r == action:
            r = int(rand()*5)
        next_pos = jumpToNextPos(pos,r)
    return r

In [237]:
def buildNetwork(X_train,y_train):
    clf = MLPClassifier(solver="adam",random_state=1,max_iter=1000,hidden_layer_sizes=(20,50,10))
    print("X_train.shape = {0},y_train.shape={1}".format(X_train.shape,y_train.shape))
    print("y_train={0}".format(y_train))
    clf.fit(X_train,y_train)
    return clf

In [238]:
def initNetwork(field):
    X_train = np.asarray([field])
    X_train = X_train.reshape(1,field_row*field_col)
    y_train = np.array([[0]])
    clf = buildNetwork(X_train,y_train)
    return clf

In [239]:
def predictAction(clf,field):
    X_test = np.asarray(field).reshape(1,field_row*field_col)
    action = clf.predict(X_test)[0]
    if rand() < 0.3:
        action = int((action + rand() * 4 + 1)) % 5
    return action

In [240]:
def createTrainData(init_pos,field,goal_pos,clf=None):
    createFlag = False
    field_data = [copy.deepcopy(field)]
    action_data = []
    global now_pos_ground
    pos = init_pos
    for i in range(SEARCH_ROOT_MAX_LOOP):
        if clf != None:
            pred_action = predictAction(clf,field)
        else:
            pred_action = int(rand()*5)
        action = checkPosition(pos,pred_action,field)
        action_data.append(action)
        next_pos = jumpToNextPos(pos,action)
#         print("next_pos = {0}".format(next_pos))
        now_pos_ground = updateField(pos,next_pos,field)
        if next_pos == goal_pos:
#             printFieldData(field_data,action_data)
            createFlag = True
            break
        field_data.append(copy.deepcopy(field))
        pos = next_pos
    return field_data,action_data,createFlag

In [241]:
def printFieldData(field_data,action_data):
    print("------------------------------------------------")
    for i in range(len(field_data)):
        print(field_data[i])
        print(action_data[i])
    print("------------------------------------------------")

In [242]:
def updateField(pos,next_pos,field):
    field[pos[0],pos[1]] = now_pos_ground
    next_pos_ground = field[next_pos[0],next_pos[1]]
    field[next_pos[0],next_pos[1]] = 2
#     print(field)
#     print("\n")
    return next_pos_ground

In [264]:
SEARCH_ROOT_MAX_LOOP = 100
def searchPath(agent_num):
    field,agents_pos_list,goal_pos_list = inputFieldData("singleAgentField2.txt")
    start_pos = agents_pos_list[agent_num]
    goal_pos = goal_pos_list[agent_num]
    print("init_field = \n{0}".format(field))
    clf = None
    path_count = 1e+7
    pathes = []
    for i in range(200):
        createFlag = False
        now_pos_ground = 0
        if (i+1) % 10 == 0:
            print("epoch = {0}".format(i+1))
        field_data,action_data,createFlag = createTrainData(start_pos,field,goal_pos,clf)
        if createFlag:
            np_field_data = np.asarray(field_data)
            np_action_data = np.asarray(action_data)
            if path_count > np_field_data.shape[0]-20:
                path_count = np_field_data.shape[0]
                clf = buildNetwork(np_field_data.reshape(np_field_data.shape[0],np_field_data.shape[1]*np_field_data.shape[2]),np_action_data)
                pathes.append([action_data,len(action_data)])
    return pathes

In [289]:
pathes_list = [searchPath(i) for i in range(len(agents_pos_list))]

init_field = 
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
X_train.shape = (42, 64),y_train.shape=(42,)
y_train=[2 0 4 2 3 0 0 1 3 0 2 2 0 0 2 2 0 4 2 0 1 2 4 3 2 1 1 4 4 1 4 4 4 0 0 0 4
 1 4 1 0 0]
epoch = 10
X_train.shape = (59, 64),y_train.shape=(59,)
y_train=[3 4 1 0 3 1 4 4 4 3 0 0 1 0 2 0 3 1 3 2 1 0 1 1 3 4 4 4 1 4 4 4 2 0 2 0 4
 4 4 4 4 2 0 2 0 4 2 4 0 4 3 1 4 2 2 1 1 0 0]
epoch = 20
epoch = 30
epoch = 40
epoch = 50
X_train.shape = (16, 64),y_train.shape=(16,)
y_train=[0 2 2 1 1 0 1 1 2 4 0 0 0 4 0 0]
epoch = 60
epoch = 70
epoch = 80
epoch = 90
epoch = 100
X_train.shape = (20, 64),y_train.shape=(20,)
y_train=[2 1 1 0 1 3 1 3 1 1 0 0 0 4 2 0 2 0 4 0]
epoch = 110
epoch = 120
epoch = 130
epoch = 140
epoch = 150
epoch = 160
epoch = 170
epoch = 180
ep

In [303]:
for (i,pathes)  in enumerate(pathes_list):
    print("agents_num = {0}".format(i+1))
    print("pathes = ")
    for path in pathes:
        print(path)
    print("\n")

#modify path
for (i,pathes) in enumerate(pathes_list):
    for path_array in pathes:
        path = path_array[0]
        del_flag = True
        print("before modifying path : \n{0}".format(path))
        while del_flag == True:
            del_flag = False
            for i in range(len(path)):
                if i+1 < len(path):
                    if path[i] == 4:
                        continue
                    if path[i]%2 == path[i+1]%2 and not(path[i] == path[i+1]):
                        del path[i+1]
                        del path[i]
                        del_flag == True
        print("after modifying path: \n{0}".format(path))
        path_array[1] = len(path)

agents_num = 1
pathes = 
[[2, 1, 1, 0, 1, 1, 0, 0, 0, 0], 10]
[[2, 1, 1, 0, 1, 1, 0, 0, 4, 0], 10]
[[4, 2, 3, 0, 0, 0, 1, 3, 2, 1, 1, 4, 4, 1, 4, 4, 4, 0, 0, 1, 4, 1, 0, 0], 24]
[[3, 4, 1, 4, 4, 3, 0, 0, 1, 0, 3, 2, 1, 0, 1, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 2, 3, 1, 4, 2, 2, 1, 1, 0, 0], 35]


agents_num = 2
pathes = 
[[1, 1, 1, 2, 2, 1, 1, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2], 17]
[[4, 4, 4, 4, 1, 4, 4, 4, 0, 1, 1, 2, 2, 1, 1, 4, 4, 4, 2], 19]
[[4, 1, 0, 1, 4, 1, 2, 2, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, 4, 4, 4, 4, 4, 4, 2], 30]
[[4, 4, 4, 1, 0, 1, 1, 4, 4, 1, 3, 4, 4, 4, 4, 4, 4, 1, 1, 4, 3, 3, 4, 1, 1, 4, 2, 2, 2, 2], 30]
[[4, 4, 4, 4, 1, 1, 4, 1, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 4, 2], 32]
[[4, 2, 1, 0, 1, 4, 4, 4, 4, 3, 3, 4, 1, 1, 4, 4, 4, 3, 3, 2, 1, 4, 0, 1, 4, 1, 2, 2, 1, 1, 4, 3, 4, 1, 0, 4, 4, 4, 2, 2, 4, 4, 4, 2], 44]
[[1, 4, 1, 1, 4, 4, 4, 4, 4, 1, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 2],

In [304]:
#sort
sorted_pathes_list = []
for pathes_array in pathes_list:
    pathes_array.sort(key=itemgetter(1))
    pathes = []
    for path_data in pathes_array:
        path = path_data[0]
#         print("path = {0}".format(path))
        pathes.append(path)
    sorted_pathes_list.append(pathes)
# for i in range(len(sorted_pathes_list[0])):
#     print(sorted_pathes_list[0][i])

In [305]:
def checkState(pos,next_pos,previous_field,field):
    if pos == next_pos:
        return True
    if field[next_pos[0],next_pos[1]] == 1 or field[next_pos[0],next_pos[1]] == 2:
        #if next_pos is reserved
        return False
    elif previous_field[next_pos[0],next_pos[1]] == 2:
        # if the agent cross other agent
        return False
    else:
        return True

In [306]:
# def updateFieldUsingPath(start_pos,pathes,first=False):
#     for j in range(len(pathes)):
#         roop_flag = False
#         print("j={0}".format(j))
#         pos = start_pos
#         path = pathes[-(j+1)]
#         print(path)
#         for i in range(len(path)):
#             next_pos = jumpToNextPos(pos,path[i])
#             if first:
#                 field_copy = copy.deepcopy(field)
#             else:
#                 if i < len(field_data):
#                     field_copy = field_data[i]
#                 else:
#                     field_copy = copy.deepcopy(field)
            
#             if checkState(next_pos,field_copy):
#                 field_copy[next_pos[0],next_pos[1]] = 2
#                 field_copy[pos[0],pos[1]] = 0
#                 if first:
#                     field_data.append(copy.deepcopy(field_copy))
                
#                 pos = next_pos
#             else:
#                 roop_flag = True
#                 break
#         if roop_flag:
#             continue
#         right_path = path
#         break
#     return right_path

In [307]:
def checkReservedAgentPath(pos):
    for (i,root_pos) in enumerate (root_pos_list):
        agent_start_pos = agents_start_pos_list[i]
#         print("agent_start_pos = {0}".format(agent_start_pos))
#         print("root_pos = {0}".format(root_pos))
        for other_agent_pos in root_pos:
            if other_agent_pos == pos:
                return False
    return True

In [308]:
def updateFieldUsingPath(start_pos,pathes):
    for i in range(len(pathes)):
        print("CCCC")
        path = pathes[-(i+1)]
        pos = start_pos
        root_pos = []
        for j in range(len(path)):
            append_flag = False
            roop_flag = False
            next_pos = jumpToNextPos(pos,path[j])
            if j < len(field_data):
                field_copy = field_data[j]
            else:
                field_copy = copy.deepcopy(field_data[-1])
                append_flag = True
            if j == 0:
                previous_field = field_data[0]
            else:
                previous_field = field_data[j-1]
            if checkState(pos,next_pos,previous_field,field_copy):
                field_copy[next_pos[0],next_pos[1]] = 2
                field_copy[pos[0],pos[1]] = 0
                root_pos.append(next_pos)
                if append_flag:
                    field_data.append(field_copy)
            else:
                print("dame : {0}".format(next_pos))
#                 print(field_copy)
                for (k,pos) in enumerate(root_pos):
                    field_data[k][pos[0],pos[1]] = 0
                roop_flag = True
                break
            pos = next_pos
            print(field_copy)
            print("")
            
        check_reserved_agent_path_flag = False
        if roop_flag == False:
            if checkReservedAgentPath(pos) == False:
                print("AAAA")
                check_reserved_agent_path_flag = True
                for (k,pos) in enumerate(root_pos):
                    field_data[k][pos[0],pos[1]] = 0
            else:
                right_path = path
                if len(right_path) < len(field_data):
                    for j in range(len(field_data)):
                        if j < len(right_path):
                            continue
                        else:
                            field_data[j][pos[0],pos[1]] = 2
                            print(field_data[j])
                return path,root_pos
        if check_reserved_agent_path_flag:
            print("BBBB")
    if i+1 == len(pathes):
        print("-----------------------------------search fault--------------------------------")
    return path,root_pos

In [311]:
field,agents_pos_list,goal_pos_list = inputFieldData("singleAgentField2.txt")
agents_start_pos_list = copy.deepcopy(agents_pos_list)
field_data = [copy.deepcopy(field)]
right_path_list = []
root_pos_list = []
for (i,pathes) in enumerate (sorted_pathes_list):
    right_path,root_pos = updateFieldUsingPath(agents_pos_list[i],pathes)
    print("right_path{0} = {1}".format(i+1,right_path))
    right_path_list.append(right_path)
    root_pos_list.append(root_pos)
    
step_num = max(len(right_path_list[0]),len(right_path_list[1]),len(right_path_list[2]))
field_data = [copy.deepcopy(field) for i in range(step_num)]
agents_num = len(agents_pos_list)
init_field = copy.deepcopy(field)
for i in range(agents_num):
    init_field[agents_pos_list[i][0],agents_pos_list[i][1]] = 2
    print("agent{0}_start_pos = {1}".format(i+1,agents_pos_list[i]))
for i in range(step_num):
    field = field_data[i]
    for j in range(agents_num):
        if i < len(right_path_list[j]):
            next_pos = jumpToNextPos(agents_pos_list[j],right_path_list[j][i])
            field[next_pos[0],next_pos[1]] = 2
            agents_pos_list[j] = next_pos
        else:
            field[agents_pos_list[j][0],agents_pos_list[j][1]] = 2
field_data.insert(0,init_field)

CCCC
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  2.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  2.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.


[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  2.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  2.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  2.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  2.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.

 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  2.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  2.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  2.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1

IndexError: index 8 is out of bounds for axis 0 with size 8

In [313]:
# for field in field_data:
#     print(field)
#     print("\n")

In [314]:
field_data_string = []
for field in field_data:
    field_string = []
    for i in range(len(field)):
        field_string_col = []
        for j in range(len(field[0])):
            if field[i][j] == 0:
                field_string_col.append("｜")
            elif field[i][j] == 1:
                field_string_col.append("■")
            elif field[i][j] == 2:
                field_string_col.append("□")
        field_string.append(field_string_col)
    field_data_string.append(field_string)

In [315]:
for i in range(len(field_data_string)):
    new_field = field_data_string[i]
    for row in new_field:
        print(row)
    print("----------")

['■', '■', '■', '■', '■', '■', '■', '■']
['■', '｜', '□', '｜', '｜', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '｜', '■', '｜', '｜', '｜', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '□', '｜', '■', '｜', '｜', '｜', '■']
['■', '｜', '□', '｜', '｜', '■', '｜', '■']
['■', '■', '■', '■', '■', '■', '■', '■']
----------
['■', '■', '■', '■', '■', '■', '■', '■']
['■', '｜', '｜', '□', '｜', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '｜', '■', '｜', '｜', '｜', '｜', '■']
['■', '□', '｜', '■', '｜', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '｜', '｜', '■']
['■', '｜', '｜', '□', '｜', '■', '｜', '■']
['■', '■', '■', '■', '■', '■', '■', '■']
----------
['■', '■', '■', '■', '■', '■', '■', '■']
['■', '｜', '｜', '｜', '□', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '□', '■', '｜', '｜', '｜', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '｜', '｜', '■']
['■', '｜', '｜', '｜', '□', '■', '｜', '■']
['■', '■', '■', '■', '■', '■', '■',

In [316]:
print(len(field_data))

25


In [317]:
for right_path in right_path_list:
    print(right_path)

[2, 1, 1, 0, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 2, 2, 1, 1, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 1, 0, 1, 4, 4, 1, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 1, 1, 2, 2, 3]
